In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from sklearn.linear_model import LinearRegression

In [2]:
df= pd.read_csv("../data/raw/data.csv")
df_dictionary = pd.read_csv('../data/external/dictionary.csv')

In [3]:
df_dictionary

,variavel,descricao,tipo,subtipo
0,carat,"Peso do diamante (0,2--5,01).",quantitativa,contínua
1,cut,"Qualidade do corte (Fair, Good, Very Good, Pre...",qualitativa,ordinal
2,color,"Cor de diamante, de D (melhor) a J (pior)",qualitativa,ordinal
3,clarity,"Clareza do diamante (I1 (pior), SI2 SI1, VS2, ...",qualitativa,ordinal
4,depth,Percentagem total de profundidade = z /média(x...,quantitativa,contínua
5,table,Largura do topo do diamante em relação ao pont...,quantitativa,contínua
6,price,"Preço em dólares dos EUA ($326-$18,823).",quantitativa,contínua
7,x,Comprimento em mm (0-10.74).,quantitativa,contínua
8,y,Largura em mm (0-58.9).,quantitativa,contínua
9,z,"Profundidade em mm (0-31,8).",quantitativa,contínua


In [4]:
target_variable = 'price'

ordinal_variables = (
    df_dictionary
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)
continuous_variables = (
    df_dictionary
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_variable])
y = df[target_variable]

In [5]:
ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='median')), # tratamento para dados faltantes
    ("encoding", OrdinalEncoder()), # codificação de variáveis
])

continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), # tratamento para dados faltantes
    ("normalization", StandardScaler()), # normalização de dados
])

In [6]:
preprocessor = ColumnTransformer([
    ("ordinal", ordinal_preprocessor, ordinal_variables), 
    ("continuous", continuous_preprocessor, continuous_variables), 
])